In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium

df=pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results.csv')
df_cycles=df[df['INSPECTION TYPE'].str.contains('Cycle',regex=False, na=False)]
df_cycles['INSPECTION DATE']=pd.to_datetime(df['INSPECTION DATE'])


C:\Users\kosta\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [2]:
df.columns

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE', 'Latitude', 'Longitude',
       'Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL',
       'NTA'],
      dtype='object')

In [3]:
df_cycles.ACTION.value_counts()

Violations were cited in the following area(s).                                                                                        306617
Establishment Closed by DOHMH.  Violations were cited in the following area(s) and those requiring immediate action were addressed.     11498
Establishment re-opened by DOHMH                                                                                                         3264
Establishment re-closed by DOHMH                                                                                                          873
No violations were recorded at the time of this inspection.                                                                               736
Name: ACTION, dtype: int64

In [4]:
CAMIS=df_cycles.CAMIS.unique()
#construct dictionary of individual inspections at establishment
rest_inspection_dates={}
for cam in CAMIS:
    rest_inspection_dates[cam]=df_cycles.loc[df_cycles['CAMIS']==cam]['INSPECTION DATE'].unique()


In [5]:
#turn each into equal length arrays of CAMIS an inspection date   
cam2=[]
date2=[]
for cam in CAMIS:
    for date in rest_inspection_dates[cam]:
        cam2.append(cam)
        date2.append(date)
#turn each into equal length arrays of CAMIS an inspection date        
print(len(cam2))
print(len(date2))

112071
112071


In [6]:
df_inspections=pd.DataFrame()
df_inspections['CAMIS']=cam2
df_inspections['DATE']=pd.to_datetime(date2)
df_inspections=df_inspections.sort_values(['CAMIS','DATE'])

In [7]:
#add columns to inspection data frame for inspection type, violations, score,

In [8]:
score=[]
for c, d in zip(df_inspections['CAMIS'],df_inspections['DATE']):
    score.append(df_cycles.loc[df_cycles['CAMIS']==c].loc[df_cycles['INSPECTION DATE']==d]['SCORE'].iloc[0])


In [9]:
df_inspections['SCORE']=score
df_inspections.head()

,CAMIS,DATE,SCORE
75428,30075445,2017-05-18,7.0
75430,30075445,2018-05-11,5.0
75431,30075445,2019-05-16,14.0
75429,30075445,2019-06-11,6.0
90427,30112340,2016-04-12,0.0


In [10]:
ins_type=[]
for c, d in zip(df_inspections['CAMIS'],df_inspections['DATE']):
    ins_type.append(df_cycles.loc[df_cycles['CAMIS']==c].loc[df_cycles['INSPECTION DATE']==d]['INSPECTION TYPE'].iloc[0])
df_inspections['INSPECTION TYPE']=ins_type
df_inspections.head()

,CAMIS,DATE,SCORE,INSPECTION TYPE
75428,30075445,2017-05-18,7.0,Cycle Inspection / Initial Inspection
75430,30075445,2018-05-11,5.0,Cycle Inspection / Initial Inspection
75431,30075445,2019-05-16,14.0,Cycle Inspection / Initial Inspection
75429,30075445,2019-06-11,6.0,Cycle Inspection / Re-inspection
90427,30112340,2016-04-12,0.0,Cycle Inspection / Initial Inspection


In [11]:
sorted(df_cycles['VIOLATION CODE'].dropna().unique())

['02A',
 '02B',
 '02C',
 '02D',
 '02E',
 '02F',
 '02G',
 '02H',
 '02I',
 '02J',
 '03A',
 '03B',
 '03C',
 '03D',
 '03E',
 '03F',
 '03G',
 '04A',
 '04B',
 '04C',
 '04D',
 '04E',
 '04F',
 '04G',
 '04H',
 '04I',
 '04J',
 '04K',
 '04L',
 '04M',
 '04N',
 '04O',
 '05A',
 '05B',
 '05C',
 '05D',
 '05E',
 '05F',
 '05H',
 '06A',
 '06B',
 '06C',
 '06D',
 '06E',
 '06F',
 '06G',
 '06H',
 '06I',
 '07A',
 '08A',
 '08B',
 '08C',
 '09A',
 '09B',
 '09C',
 '10A',
 '10B',
 '10C',
 '10D',
 '10E',
 '10F',
 '10G',
 '10H',
 '10I',
 '10J',
 '17A1',
 '17A3',
 '19A1',
 '19A3',
 '22F',
 '22G']

Full list of codes and descriptions is available in the "blue book" https://www1.nyc.gov/assets/doh/downloads/pdf/rii/blue-book.pdf . Some of these codes are not in that list and all of those that are not valid codes are missing the Violation Description as well. These are dropped from the dataset.

In [12]:
for v in sorted(df_cycles['VIOLATION CODE'].dropna().unique())[-6:]:
    print(len(df_cycles.loc[df_cycles['VIOLATION CODE']==v]['VIOLATION DESCRIPTION']))

2
1
1
1
427
712


In [13]:
vio_codes=sorted(df_cycles['VIOLATION CODE'].dropna().unique())[:-6]

In [ ]:
for v in vio_codes:
    temp=[]
    for c, d in zip(df_inspections['CAMIS'],df_inspections['DATE']):
        #sum of true and falses. should only every be 1 or 0
        temp.append(sum(df_cycles.loc[df_cycles['CAMIS']==c].loc[df_cycles['INSPECTION DATE']==d]['VIOLATION CODE'].str.contains(v))==1)
        
    df_inspections[v]=temp

In [ ]:
df_inspections.head()

In [ ]:
sum(df_cycles.loc[df_cycles['CAMIS']==c].loc[df_cycles['INSPECTION DATE']==d]['VIOLATION CODE'].str.contains('06D'))